In [15]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
ratings = pd.read_csv('ratings.csv', index_col=1)
ratings = ratings.iloc[:1500000]
ratings

,userId,rating,timestamp
movieId,,,
110,1,1.0,1425941529
147,1,4.5,1425942435
858,1,5.0,1425941523
1221,1,5.0,1425941546
1246,1,5.0,1425941556
...,...,...,...
3052,15424,4.0,1370209959
3147,15424,3.5,1369586045
3175,15424,3.0,1372541792


In [17]:
user_ratings = ratings.pivot_table(values='rating', columns=['movieId'], index=['userId'])

user_ratings = user_ratings.dropna(thresh=9, axis=1)
user_ratings = user_ratings.fillna(0)
user_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,168612,168712,169864,169982,169984,170705,170875,171763,173145,174055
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15420,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15421,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
del ratings
def standardise(row):
    row = (row - row.mean())/(row.max()-row.min())
    return row

ratings_std = user_ratings.apply(standardise)
ratings_std

movieId,1,2,3,4,5,6,7,8,9,10,...,168612,168712,169864,169982,169984,170705,170875,171763,173145,174055
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.193588,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
2,-0.193588,-0.062824,-0.034777,-0.005453,0.56499,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
3,-0.193588,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
4,-0.193588,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
5,-0.193588,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,0.561028,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15420,-0.193588,-0.062824,0.765223,-0.005453,0.56499,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
15421,0.406412,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985
15422,-0.193588,-0.062824,-0.034777,-0.005453,-0.03501,-0.077963,-0.038972,-0.003689,-0.009946,-0.082417,...,-0.001653,-0.000478,-0.000872,-0.000648,-0.001131,-0.000992,-0.000922,-0.00107,-0.000497,-0.000985


In [19]:
item_sim = cosine_similarity(ratings_std.T)
item_sim

array([[1.        , 0.28962342, 0.20012   , ..., 0.03218925, 0.03525562,
        0.01920127],
       [0.28962342, 1.        , 0.16335274, ..., 0.01802506, 0.01744877,
        0.01741424],
       [0.20012   , 0.16335274, 1.        , ..., 0.01372849, 0.02169059,
        0.00734735],
       ...,
       [0.03218925, 0.01802506, 0.01372849, ..., 1.        , 0.42249691,
        0.26229983],
       [0.03525562, 0.01744877, 0.02169059, ..., 0.42249691, 1.        ,
        0.28171481],
       [0.01920127, 0.01741424, 0.00734735, ..., 0.26229983, 0.28171481,
        1.        ]])

In [20]:
item_sim_df = pd.DataFrame(item_sim, index=user_ratings.columns, columns=user_ratings.columns)
del user_ratings
del item_sim
item_sim_df

movieId,1,2,3,4,5,6,7,8,9,10,...,168612,168712,169864,169982,169984,170705,170875,171763,173145,174055
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.289623,0.200120,0.044638,0.207190,0.240149,0.197007,0.060388,0.104628,0.216165,...,0.034321,0.011675,0.034804,0.027388,0.046193,0.040753,0.024120,0.032189,0.035256,0.019201
2,0.289623,1.000000,0.163353,0.136058,0.177693,0.191088,0.152193,0.144263,0.104379,0.325641,...,0.063135,0.038527,0.071282,0.065913,0.051765,0.032585,0.065394,0.018025,0.017449,0.017414
3,0.200120,0.163353,1.000000,0.102999,0.417730,0.202698,0.338935,0.080511,0.201631,0.109659,...,0.003741,0.001204,0.006329,0.006706,0.014469,0.020794,0.009545,0.013728,0.021691,0.007347
4,0.044638,0.136058,0.102999,1.000000,0.138489,0.082143,0.135361,0.188071,0.098186,0.081565,...,-0.004371,-0.002201,-0.003046,0.019182,-0.003415,-0.003031,-0.003173,-0.003291,-0.002188,-0.003094
5,0.207190,0.177693,0.417730,0.138489,1.000000,0.152526,0.359628,0.097355,0.201644,0.099962,...,-0.001527,-0.005592,-0.002258,0.013432,0.008055,0.007826,0.007229,-0.008360,0.005487,0.003661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170705,0.040753,0.032585,0.020794,-0.003031,0.007826,0.034163,0.011042,-0.002383,-0.004120,0.016345,...,0.053375,0.143264,0.050260,-0.000956,0.177856,1.000000,0.114452,0.051124,0.082397,0.046342
170875,0.024120,0.065394,0.009545,-0.003173,0.007229,0.032363,0.009134,0.015993,0.026178,0.014760,...,0.169470,0.177722,0.166396,0.286834,0.197015,0.114452,1.000000,0.071485,0.184218,-0.001170
171763,0.032189,0.018025,0.013728,-0.003291,-0.008360,0.016465,-0.008573,-0.002587,0.003704,0.012869,...,0.129042,-0.000863,0.140986,0.095954,0.184526,0.051124,0.071485,1.000000,0.422497,0.262300


In [21]:
item_sim_df.to_csv('cos_similarity_upd.csv')